In [33]:
import os
from pathlib import Path
import pandas as pd
import json
from epjson.epjson_idf import EpJsonIDF
from epjson.simulate_cluster import SimulateCluster

def run_single_building(
        config_dict, 
        output_dir, 
        weight_map,
        resource_dir=Path("tests/resources"), 
        idf_dir=Path("tests/resources/idfs/"),
        eplus_loc=Path("C:\EnergyPlusV22-2-0"), 
        update_epsjon=True
        ):
    
    weights_df = pd.read_json(resource_dir / "shoebox-weights.json")
    weights_df = weights_df[weights_df["building_id"] == config_dict["building_id"]]
    idfs = set(weights_df.ShoeboxPath.to_list())

    ### MUTATIONS ###

    # Get retrofit scenario template data
    scenario_dir = resource_dir / f"templates_RETRO_{config_dict['retrofit_scenario']}_SCHED_{config_dict['schedules_scenario']}_LAB_{config_dict['lab_scenario']}.csv"
    features_df = pd.read_csv(scenario_dir)
    print(scenario_dir)
    with open(resource_dir / "template_features_map.json", "r") as f:
        features_map = json.load(f)
    with open(resource_dir / "schedule_bunches.json", "r") as f:
        schedules = json.load(f)
        
    if update_epsjon:
        for i, idf in enumerate(idfs):
            try:
                idf_path = idf_dir / idf
                epjson = EpJsonIDF.from_epjson(idf_path.with_suffix(".epjson"), eplus_loc=eplus_loc)
                features_series = features_df[(features_df.TYPOLOGY == config_dict["TYPOLOGY"]) & (features_df.AGE_ERA == config_dict["AGE_ERA"])].reset_index(drop=True)
                features_series = features_series.iloc[0]
                # Set DHW flow rate based on sb area
                peak_dhw_pp = features_series["DHWFlowRate"]
                occupants = features_series["PeopleDensity"] * (weights_df.iloc[i]["core_area"])+(weights_df.iloc[i]["perim_area"])
                features_series["DHWFlowRate"] = peak_dhw_pp * occupants

                cols = set(features_series.index) & set(features_map.keys())

                epjson.replace_features(features_series[cols], features_map)
                epjson.update_schedules(schedules)
                epjson.add([(
                    "Output:SQLite",
                    {"Output:SQLite 1": {"option_type": "Simple"}}
                    )])
                epjson.epjson["ZoneInfiltration:DesignFlowRate"]["CoreInfiltration"].update(
                    {"air_changes_per_hour": 0}
                )
                epjson.epjson["WaterUse:Equipment"]["DHW Perimeter"].update(
                    {"peak_flow_rate": 0}
                )
                
                # new_path = replace_shoebox_path(idf_path).with_suffix(".epJSON")
                new_path = output_dir / Path(idf).with_suffix(".epJSON")
                epjson.save_as(new_path)
                epjson.save_idf(output_path=new_path.parent) #suffix="_test",

            except Exception as e:
                print(f"FAILED ON {idf}")
                raise e
    
    ### SIMULATION ###
    # Get new paths for updated shoeboxes
    # new_idfs = weights_df["ShoeboxPath"].apply(replace_shoebox_path)
    new_idfs = weights_df["ShoeboxPath"].apply(lambda x: output_dir / Path(x))
    weights_df["ShoeboxPath"] = new_idfs
    
    print("BUILDING CLUSTER")
    idf_cluster = SimulateCluster(
        idf_list = set(new_idfs),
        epw = config_dict["epw"],
        weights_df = weights_df,
        building_col = "building_id",
        eplus_location=eplus_loc,
        weight_map = weight_map
    )
    print("SIMULATING CLUSTER")
    errors = idf_cluster.parallel_simulate() # Fix errors dict

    print("PROCESSING CLUSTER RESULTS")
    ### POST-PROCESS ###
    r = idf_cluster.fetch_building_results_parallel([config_dict["building_id"]])

    return r, errors, idf_cluster

In [ ]:
config_dict = {
    "building_id": "62",
    "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\BaselineFile.epw",
    "retrofit_scenario": "baseline",
    "schedules_scenario": "baseline",
    "lab_scenario": "baseline",
    "TYPOLOGY": "Residential",
    "AGE_ERA": 1945,
    "lab_weight": 0.268159426
}

sb_weight_map = {
    "Perimeter": "PerimeterAreaWeight",
    "Core": "CoreAreaWeight"
}

office_res, office_err, office_cluster = run_single_building(config_dict, 
                               weight_map = sb_weight_map,
                               output_dir = Path(os.getcwd()) / "eplus", 
                               idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                               update_epsjon=True
                               )

In [44]:
office_res[config_dict["building_id"]].sum()

C:\Users\zoelh\AppData\Local\Temp\ipykernel_575388\1685267888.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  office_res[config_dict["building_id"]].sum()


Zone Lights Electricity Energy                                                               197316.252678
Electric Equipment Electricity Energy                                                        403749.280475
Zone Ideal Loads Supply Air Total Heating Energy                                             349083.670209
Zone Ideal Loads Supply Air Total Cooling Energy                                              537448.82727
Water Use Equipment Heating Energy                                                            413424.51759
Zone Lights Electricity Energy_norm                                                               15.11091
Electric Equipment Electricity Energy_norm                                                       30.920002
Zone Ideal Loads Supply Air Total Heating Energy_norm                                             26.73359
Zone Ideal Loads Supply Air Total Cooling Energy_norm                                            41.159005
Water Use Equipment Heating Energy_no

Run all archetypes

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
res = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        # "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\BaselineFile.epw",
        "retrofit_scenario": "baseline",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    res[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [48]:
norm_cols = ['Zone Lights Electricity Energy_norm',
       'Electric Equipment Electricity Energy_norm',
       'Zone Ideal Loads Supply Air Total Heating Energy_norm',
       'Zone Ideal Loads Supply Air Total Cooling Energy_norm',
       'Water Use Equipment Heating Energy_norm']
pd.DataFrame.from_dict(res).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                            174.709437   
                     1980 W31                           152.870765   
Lab and Mixed Use    1945 1                             104.825664   
                     1980 13                            104.825662   
                     2015 12                            104.825661   
Office and Mixed Use 1945 50                            104.825659   
                     1980 W11                            50.665741   
Residential          1945 62                              15.11091   
                     1980 NW30                           15.110911   
                     2015 E37                             15.11091   
Support Dominant     1945 42                             26.570394   
                     1980 E23                            26.570394   
                     2015 42C                            26.570393   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                                  0.0      
                     1980 W31                                            0.002605      
Lab and Mixed Use    1945 1                                              1.366121      
                     1980 13                                             0.544421      
                     2015 12                                             0.946652      
Office and Mixed Use 1945 50                                             0.913093      
                     1980 W11                                            2.468519      
Residential          1945 62                                            41.777946      
                     1980 NW30                                          19.554971      
                     2015 E37                                            6.156634      
Support Dominant     1945 42                                            15.474506      
                     1980 E23                                           18.743013      
                     2015 42C                                             5.47735      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            73.146775      
                     1980 W31                                           68.268488      
Lab and Mixed Use    1945 1                                            544.486723      
                     1980 13                                           441.867587      
                     2015 12                                           230.790547      
Office and Mixed Use 1945 50                                             542.1193      
                     1980 W11                                          367.229697      
Residential          1945 62      

In [16]:
norm_cols = ['Zone Lights Electricity Energy_norm',
       'Electric Equipment Electricity Energy_norm',
       'Zone Ideal Loads Supply Air Total Heating Energy_norm',
       'Zone Ideal Loads Supply Air Total Cooling Energy_norm',
       'Water Use Equipment Heating Energy_norm']
pd.DataFrame.from_dict(res).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                            174.709437   
                     1980 W31                           152.870765   
Lab and Mixed Use    1945 1                             104.825664   
                     1980 13                            104.825662   
                     2015 12                            104.825661   
Office and Mixed Use 1945 50                            104.825659   
                     1980 W11                            50.665741   
Residential          1945 62                              31.75643   
                     1980 NW30                           31.756433   
                     2015 E37                            25.405145   
Support Dominant     1945 42                             26.570394   
                     1980 E23                            26.570394   
                     2015 42C                            26.570393   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    44.683732   
                     1980 NW30                                  44.683736   
                     2015 E37                                   35.746987   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                            11.502478      
                     1980 W31                                            0.132788      
Lab and Mixed Use    1945 1                                             52.011467      
                     1980 13                                            19.896902      
                     2015 12                                            35.594914      
Office and Mixed Use 1945 50                                            34.958461      
                     1980 W11                                           50.830942      
Residential          1945 62                                            89.410137      
                     1980 NW30                                          54.867065      
                     2015 E37                                            43.13459      
Support Dominant     1945 42                                            37.592383      
                     1980 E23                                           43.722336      
                     2015 42C                                           20.453336      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                             78.29089      
                     1980 W31                                           70.810904      
Lab and Mixed Use    1945 1                                            411.118182      
                     1980 13                                             344.5176      
                     2015 12                                           184.978676      
Office and Mixed Use 1945 50                                           375.643906      
                     1980 W11                                          209.916606      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
partialres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "partial",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    partialres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [9]:
pd.DataFrame.from_dict(partialres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              52.412832   
                     1980 13                             52.412831   
                     2015 12                              52.41283   
Office and Mixed Use 1945 50                              52.41283   
                     1980 W11                             25.33287   
Residential          1945 62                             15.878215   
                     1980 NW30                           15.878217   
                     2015 E37                            12.702573   
Support Dominant     1945 42                             13.285197   
                     1980 E23                            13.285197   
                     2015 42C                            13.285196   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   415.191115   
                     1980 W31                                  148.299103   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   124.743663   
                     2015 12                                    32.231307   
Office and Mixed Use 1945 50                                    174.68143   
                     1980 W11                                  209.858257   
Residential          1945 62                                    34.503715   
                     1980 NW30                                  19.112444   
                     2015 E37                                   18.329249   
Support Dominant     1945 42                                    70.262852   
                     1980 E23                                   68.219383   
                     2015 42C                                   57.525876   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                             0.548628      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                               11.4577      
                     1980 13                                             0.577836      
                     2015 12                                             7.350539      
Office and Mixed Use 1945 50                                              0.80183      
                     1980 W11                                            8.928416      
Residential          1945 62                                            80.290397      
                     1980 NW30                                          51.132714      
                     2015 E37                                           45.118812      
Support Dominant     1945 42                                             17.22968      
                     1980 E23                                           20.085042      
                     2015 42C                                            8.899019      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            64.294289      
                     1980 W31                                           29.331825      
Lab and Mixed Use    1945 1                                            109.305124      
                     1980 13                                            54.185818      
                     2015 12                                            45.161389      
Office and Mixed Use 1945 50                                            66.683178      
                     1980 W11                                           43.139863      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
fullres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "full",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    fullres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [11]:
pd.DataFrame.from_dict(fullres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              52.412832   
                     1980 13                             52.412831   
                     2015 12                              52.41283   
Office and Mixed Use 1945 50                              52.41283   
                     1980 W11                             25.33287   
Residential          1945 62                             15.878215   
                     1980 NW30                           15.878217   
                     2015 E37                            12.702573   
Support Dominant     1945 42                             13.285197   
                     1980 E23                            13.285197   
                     2015 42C                            13.285196   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   415.191115   
                     1980 W31                                  148.299103   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   124.743663   
                     2015 12                                    32.231307   
Office and Mixed Use 1945 50                                    174.68143   
                     1980 W11                                  209.858257   
Residential          1945 62                                    34.503715   
                     1980 NW30                                  19.112444   
                     2015 E37                                   18.329249   
Support Dominant     1945 42                                    70.262852   
                     1980 E23                                   68.219383   
                     2015 42C                                   57.525876   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                                  0.0      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                              1.183078      
                     1980 13                                             0.004149      
                     2015 12                                             3.680891      
Office and Mixed Use 1945 50                                             0.001412      
                     1980 W11                                             3.66378      
Residential          1945 62                                            49.078417      
                     1980 NW30                                          31.202922      
                     2015 E37                                           33.868399      
Support Dominant     1945 42                                             6.424031      
                     1980 E23                                            5.572564      
                     2015 42C                                            3.722225      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            63.446566      
                     1980 W31                                           30.014403      
Lab and Mixed Use    1945 1                                            108.195109      
                     1980 13                                            55.048434      
                     2015 12                                            46.171233      
Office and Mixed Use 1945 50                                            68.141578      
                     1980 W11                                           45.527958      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
allfullres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "full",
        "schedules_scenario": "full",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    allfullres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()


In [13]:
pd.DataFrame.from_dict(allfullres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              32.648827   
                     1980 13                             32.648826   
                     2015 12                             32.648826   
Office and Mixed Use 1945 50                             32.648825   
                     1980 W11                            32.648829   
Residential          1945 62                             19.472822   
                     1980 NW30                           19.472824   
                     2015 E37                            15.578258   
Support Dominant     1945 42                             27.207356   
                     1980 E23                            27.207355   
                     2015 42C                            27.207354   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   415.191115   
                     1980 W31                                  148.299103   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   124.743663   
                     2015 12                                    32.231307   
Office and Mixed Use 1945 50                                    174.68143   
                     1980 W11                                  209.858257   
Residential          1945 62                                    34.503715   
                     1980 NW30                                  19.112444   
                     2015 E37                                   18.329249   
Support Dominant     1945 42                                    70.262852   
                     1980 E23                                   68.219383   
                     2015 42C                                   57.525876   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                             0.004001      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                              0.634639      
                     1980 13                                              0.03204      
                     2015 12                                             1.630707      
Office and Mixed Use 1945 50                                             0.006834      
                     1980 W11                                            1.325767      
Residential          1945 62                                            32.262924      
                     1980 NW30                                           19.12792      
                     2015 E37                                           20.730034      
Support Dominant     1945 42                                             1.394297      
                     1980 E23                                            1.068332      
                     2015 42C                                            0.572345      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            64.067286      
                     1980 W31                                           29.570941      
Lab and Mixed Use    1945 1                                             82.508103      
                     1980 13                                            40.181428      
                     2015 12                                            32.672989      
Office and Mixed Use 1945 50                                            48.383655      
                     1980 W11                                           36.061293      
Residential          1945 62      